# Penjadwalan Makanan Sugar Glider dengan Algoritma Genetika

Program ini mengoptimalkan jadwal makanan mingguan untuk Sugar Glider menggunakan Algoritma Genetika dengan mempertimbangkan kebutuhan nutrisi yang spesifik.

In [95]:
# Import library yang diperlukan
import random
import math
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from datetime import datetime
import numpy as np
import copy
from IPython.display import display # Untuk menampilkan DataFrame di Jupyter

In [ ]:
# ==============================================================================
# I. LOAD DATA DAN PRE-PROCESSING
# ==============================================================================

# Load data makanan dan proses indeks kategori
data_makanan = pd.read_csv('data/data_makanan.csv')
print(f"Data makanan berhasil dimuat dari data/data_makanan.csv.")


# Create output directory if it doesn't exist
output_dir = './output'
os.makedirs(output_dir, exist_ok=True)

# Add kalori and berat columns if missing (estimated values)
if 'kalori' not in data_makanan.columns:
    data_makanan['kalori'] = (data_makanan['protein'] * 4 + 
                              data_makanan['gula'] * 4).round(1)

if 'berat' not in data_makanan.columns:
    def get_standard_weight(jenis):
        jenis = str(jenis).strip().lower()
        if jenis == 'protein': return 10.0
        elif jenis == 'buah': return 50.0
        elif jenis == 'sayur': return 30.0
        elif jenis == 'supp': return 0.1
        else: return 10.0
    data_makanan['berat'] = data_makanan['jenis'].apply(get_standard_weight)

# Process data and create indices
foods = []
protein_idx = []
fruit_idx = []
veg_idx = []
supp_idx = []

for i, row in data_makanan.iterrows():
    try:
        # Konversi dan normalisasi data nutrisi
        foods.append([
            row['nama_makanan'],     # 0: nama
            row['jenis'],           # 1: kategori
            float(row['protein']) if pd.notna(row['protein']) else 0.0,      # 2: protein (g)
            float(row['serat']) if pd.notna(row['serat']) else 0.0,          # 3: serat (g)
            float(row['kalsium'])/1000 if pd.notna(row['kalsium']) else 0.0, # 4: kalsium (convert mg to g)
            float(row['fosfor'])/1000 if pd.notna(row['fosfor']) else 0.0,   # 5: fosfor (convert mg to g)
            float(row['kalori']) if pd.notna(row['kalori']) else 0.0,        # 6: kalori (kcal)
            float(row['gula']) if pd.notna(row['gula']) else 0.0,            # 7: gula (g)
            float(row['berat']) if pd.notna(row['berat']) else 10.0          # 8: berat saji (g)
        ])
        
        jenis = str(row['jenis']).strip().lower()
        
        if jenis == 'protein': protein_idx.append(i)
        elif jenis == 'buah': fruit_idx.append(i)
        elif jenis == 'sayur': veg_idx.append(i)
        elif jenis == 'supp': supp_idx.append(i)
    
    except Exception as e:
        print(f"Error processing row {i}: {e}")
        continue

# Check data readiness
if not all([protein_idx, fruit_idx, veg_idx]):
    print("\nFATAL ERROR: Kategori makanan tidak lengkap. Optimasi tidak dapat dilakukan.")
    # Jika menggunakan data dummy dan ingin agar kode tetap jalan, hapus 'exit()' atau berikan indeks dummy
    # exit()

print(f"Data makanan berhasil dimuat: {len(foods)} items")
print(f"Kategori: Protein: {len(protein_idx)}, Buah: {len(fruit_idx)}, Sayuran: {len(veg_idx)}, Suplemen: {len(supp_idx)}")
print("-" * 50)

Data makanan berhasil dimuat dari data/data_makanan.csv.
Data makanan berhasil dimuat: 36 items
Kategori: Protein: 9, Buah: 10, Sayuran: 10, Suplemen: 3
--------------------------------------------------


In [97]:
# ==============================================================================
# II. DEFINISI ALGORITMA GENETIKA (AG)
# ==============================================================================

## 1. Individu dan Inisialisasi
# Representasi: list[ 7 hari ], tiap hari = [protein_idx, fruit_idx, veg_idx, supplement_status (0/1)]

def random_day():
    """Membuat kromosom hari acak."""
    return [
        random.choice(protein_idx),
        random.choice(fruit_idx),
        random.choice(veg_idx),
        random.choice([0,1])  # status suplemen
    ]

def random_individual():
    """Membuat individu (jadwal 7 hari) acak."""
    return [random_day() for _ in range(7)]

## 2. Fungsi Hitung Nutrisi

def compute_day_nutrition(day):
    """Menghitung total nutrisi harian dari satu hari jadwal."""
    prot_idx, fru_idx, veg_idx, sup_status = day
    items = [foods[prot_idx], foods[fru_idx], foods[veg_idx]]
    
    # Tambahkan suplemen jika status = 1 dan ada data suplemen
    if sup_status == 1 and len(supp_idx) > 0:
        items.append(foods[supp_idx[0]])

    # Indeks Nutrisi: 2=P, 3=S, 4=Ca, 5=Phos, 6=Kcal, 7=Gula, 8=Mass
    protein = sum(i[2] for i in items)
    fiber   = sum(i[3] for i in items)
    ca      = sum(i[4] for i in items)
    phos    = sum(i[5] for i in items)
    kcal    = sum(i[6] for i in items)
    sugar   = sum(i[7] for i in items)
    mass    = sum(i[8] for i in items)

    return protein, fiber, ca, phos, kcal, sugar, mass

## 3. Fungsi Fitness

def fitness(ind):
    """Menghitung seberapa baik (fitness) suatu jadwal mingguan."""
    total = [compute_day_nutrition(d) for d in ind]
    total_p = sum(x[0] for x in total)
    total_f = sum(x[1] for x in total)
    total_ca = sum(x[2] for x in total)
    total_ph = sum(x[3] for x in total)
    total_kcal = sum(x[4] for x in total)
    total_sugar = sum(x[5] for x in total)
    total_mass = sum(x[6] for x in total)

    # Guard: Hindari pembagian dengan nol
    if total_kcal == 0 or total_ph == 0 or total_mass == 0:
        return 1e-6 

    # Hitung metrik nutrisi mingguan
    protein_pct = (4*total_p)/total_kcal*100   
    fiber_pct = (total_f/total_mass)*100       
    ca_phos_ratio = total_ca/total_ph          
    sugar_pct = (4*total_sugar)/total_kcal*100 
    avg_kcal = total_kcal / 7                  

    # --- Hitung Skor Komponen ---

    # 1. Protein Score (Target 30%)
    score_p = max(0, 1 - abs(protein_pct - 30)/30)

    # 2. Fiber Score (Target 35% dari massa)
    score_f = max(0, 1 - abs(fiber_pct - 35)/35)

    # 3. Ca:P Ratio Score (Target 2:1) - Gaussian
    score_ca = math.exp(-((ca_phos_ratio - 2)**2) / (2 * 0.5**2))

    # 4. Sugar Score (Target <= 10%)
    score_s = 1 if sugar_pct <= 10 else max(0, 1 - (sugar_pct - 10)/20)

    # 5. Calorie Score (Target 65 kcal/hari)
    score_k = 1 if 50 <= avg_kcal <= 80 else max(0, 1 - abs(avg_kcal - 65)/100)

    # 6. Variety Score (Variasi sumber protein)
    prot_sources = [d[0] for d in ind]
    variety = len(set(prot_sources))/len(ind)
    score_v = variety

    # --- Final Score dengan Bobot ---
    final_score = (0.25*score_p + 0.20*score_f + 0.25*score_ca + 
                   0.10*score_s + 0.10*score_k + 0.10*score_v)

    return final_score

## 4. Operator Genetika (Crossover, Mutasi, Seleksi)

def crossover(parent1, parent2):
    """Single point crossover pada hari."""
    point = random.randint(1, len(parent1) - 1)
    child1 = parent1[:point] + parent2[point:]
    child2 = parent2[:point] + parent1[point:]
    return child1, child2

def mutate(individual, mutation_rate):
    """Mutasi sederhana."""
    for day in individual:
        if random.random() < mutation_rate and protein_idx:
            day[0] = random.choice(protein_idx)
        if random.random() < mutation_rate and fruit_idx:
            day[1] = random.choice(fruit_idx)
        if random.random() < mutation_rate and veg_idx:
            day[2] = random.choice(veg_idx)
        if random.random() < mutation_rate:
            day[3] = random.choice([0, 1])

def tournament_selection_pair(population, fitnesses, k=5):
    """Tournament Selection untuk memilih 2 parent."""
    pop_fit = list(zip(population, fitnesses))
    parents = []
    
    for _ in range(2):
        tournament = random.sample(pop_fit, k)
        winner = max(tournament, key=lambda x: x[1])[0]
        parents.append(winner)
        
    return parents[0], parents[1]

In [98]:
# ==============================================================================
# III. EKSEKUSI ALGORITMA GENETIKA
# ==============================================================================
POP_SIZE = 90
GENERATIONS = 200
MUTATION_RATE = 0.05
CROSSOVER_RATE = 1.0

# Track history
best_fitness_history = []
avg_fitness_history = []
best_individual = None

# Inisialisasi populasi
population = [random_individual() for _ in range(POP_SIZE)] 

print("="*60)
print("ALGORITMA GENETIKA UNTUK PENJADWALAN MAKANAN")
print(f"Populasi: {POP_SIZE}, Generasi: {GENERATIONS}, Mutasi Rate: {MUTATION_RATE}")
print("="*60)

for generation in range(GENERATIONS):
    # Evaluasi fitness
    fitnesses = [fitness(ind) for ind in population]
    
    # Track history
    best_fit = max(fitnesses)
    avg_fit = sum(fitnesses) / len(fitnesses)
    best_fitness_history.append(best_fit)
    avg_fitness_history.append(avg_fit)
    
    # Simpan individu terbaik global
    current_best_ind = population[fitnesses.index(best_fit)]
    if best_individual is None or fitness(best_individual) < best_fit:
        best_individual = copy.deepcopy(current_best_ind)

    # Print progress
    if generation % 20 == 0 or generation == GENERATIONS - 1:
        print(f"Gen {generation:3d}: Best={best_fit:.4f}, Avg={avg_fit:.4f}")
    
    # Buat populasi baru
    new_population = []
    
    # 1. Elitism: simpan 2 individu terbaik
    sorted_pop = sorted(zip(population, fitnesses), key=lambda x: x[1], reverse=True)
    
    new_population.append(copy.deepcopy(sorted_pop[0][0]))
    new_population.append(copy.deepcopy(sorted_pop[1][0]))
    
    # 2. Buat offspring sampai populasi penuh
    while len(new_population) < POP_SIZE:
        # Selection
        parent1, parent2 = tournament_selection_pair(population, fitnesses)
        
        # Crossover
        if random.random() < CROSSOVER_RATE:
            child1, child2 = crossover(parent1[:], parent2[:]) 
        else:
            child1, child2 = parent1[:], parent2[:]
        
        # Mutation
        mutate(child1, MUTATION_RATE)
        mutate(child2, MUTATION_RATE)
        
        new_population.append(child1)
        if len(new_population) < POP_SIZE:
            new_population.append(child2)
    
    population = new_population

final_best_fitness = fitness(best_individual)
print(f"\nEvolusi selesai! Fitness terbaik global: {final_best_fitness:.4f}")
print("="*60)

ALGORITMA GENETIKA UNTUK PENJADWALAN MAKANAN
Populasi: 90, Generasi: 200, Mutasi Rate: 0.05
Gen   0: Best=0.3554, Avg=0.2461
Gen  20: Best=0.4078, Avg=0.2997
Gen  40: Best=0.4089, Avg=0.3001
Gen  60: Best=0.4121, Avg=0.3237
Gen  80: Best=0.4169, Avg=0.3071
Gen 100: Best=0.4204, Avg=0.3299
Gen 120: Best=0.4340, Avg=0.3077
Gen 140: Best=0.4351, Avg=0.3187
Gen 160: Best=0.4433, Avg=0.3283
Gen 180: Best=0.4433, Avg=0.3100
Gen 199: Best=0.4433, Avg=0.3471

Evolusi selesai! Fitness terbaik global: 0.4433


In [99]:
# ==============================================================================
# IV. ANALISIS DAN OUTPUT HASIL TERBAIK
# ==============================================================================

best = best_individual
days = ["Senin","Selasa","Rabu","Kamis","Jumat","Sabtu","Minggu"]

# Create detailed schedule & nutrition data
result = []
nutrition_per_day = []

for i, day in enumerate(best):
    prot_idx, fru_idx, veg_idx, sup_status = day
    day_nutrition = compute_day_nutrition(day)
    
    result.append([
        days[i],
        foods[prot_idx][0],
        foods[fru_idx][0],
        foods[veg_idx][0],
        "Ya" if sup_status==1 else "Tidak"
    ])
    
    nutrition_per_day.append({
        'Hari': days[i],
        'Protein (g)': day_nutrition[0],
        'Serat (g)': day_nutrition[1],
        'Kalsium (mg)': day_nutrition[2] * 1000, # Konversi kembali ke mg
        'Fosfor (mg)': day_nutrition[3] * 1000,   # Konversi kembali ke mg
        'Kalori (kcal)': day_nutrition[4],
        'Gula (g)': day_nutrition[5],
        'Massa Total (g)': day_nutrition[6]
    })

# Create DataFrames
df_schedule = pd.DataFrame(result, columns=["Hari","Protein","Buah","Sayur","Suplemen Kalsium"])
df_nutrition = pd.DataFrame(nutrition_per_day)

# Calculate totals and percentages for summary
total_nutrition = df_nutrition.sum(numeric_only=True)
total_kcal = total_nutrition['Kalori (kcal)']
total_mass = total_nutrition['Massa Total (g)']

protein_pct = (4 * total_nutrition['Protein (g)']) / total_kcal * 100
fiber_pct = (total_nutrition['Serat (g)'] / total_mass) * 100
ca_phos_ratio = (total_nutrition['Kalsium (mg)']/1000) / (total_nutrition['Fosfor (mg)']/1000)
sugar_pct = (4 * total_nutrition['Gula (g)']) / total_kcal * 100
protein_variety_count = len(set([foods[day[0]][0] for day in best]))

# Create summary report
summary_data = {
    'Metrik': [
        'Total Kalori (7 hari)', 'Total Protein (g)', 'Total Serat (g)',
        'Rasio Ca:P', 'Persentase Protein (%)', 'Persentase Serat (%)',
        'Persentase Gula (%)', 'Variasi Protein', 'Fitness Score'
    ],
    'Nilai': [
        f"{total_kcal:.1f} kcal", f"{total_nutrition['Protein (g)']:.2f} g", f"{total_nutrition['Serat (g)']:.2f} g",
        f"{ca_phos_ratio:.2f}:1", f"{protein_pct:.1f}%", f"{fiber_pct:.1f}%",
        f"{sugar_pct:.1f}%", f"{protein_variety_count}/7 jenis", f"{final_best_fitness:.4f}"
    ],
    'Target/Status': [
        "Target Rata-rata 65 kcal/hari", "~30% dari kalori", "~35% dari massa",
        "2:1 (optimal)", "30% (target)", "35% (target)",
        "≤10% (target)", "Maksimal 7/7", "Maksimal 1.0"
    ]
}

df_summary = pd.DataFrame(summary_data)

# DISPLAY RESULTS
print("JADWAL MAKANAN OPTIMAL UNTUK SUGAR GLIDER")
print("="*60)
display(df_schedule)

print("\nRINGKASAN NUTRISI MINGGUAN")
print("="*60)
display(df_summary)

print("\nNUTRISI HARIAN DETAIL")
print("="*60)
display(df_nutrition.round(2))

JADWAL MAKANAN OPTIMAL UNTUK SUGAR GLIDER


,Hari,Protein,Buah,Sayur,Suplemen Kalsium
0,Senin,Yogurt Tawar,Anggur (Batasi),Wortel,Ya
1,Selasa,Tahu (Matang),Anggur (Batasi),Wortel,Ya
2,Rabu,Tahu (Matang),Pir,Wortel,Ya
3,Kamis,Ikan Salmon (Matang),Pir,Wortel,Ya
4,Jumat,Telur Rebus,Pisang,Wortel,Tidak
5,Sabtu,Jangkrik,Pepaya,Wortel,Ya
6,Minggu,Yogurt Tawar,Pisang,Wortel,Tidak



RINGKASAN NUTRISI MINGGUAN


,Metrik,Nilai,Target/Status
0,Total Kalori (7 hari),1749.6 kcal,Target Rata-rata 65 kcal/hari
1,Total Protein (g),131.40 g,~30% dari kalori
2,Total Serat (g),134.80 g,~35% dari massa
3,Rasio Ca:P,91.21:1,2:1 (optimal)
4,Persentase Protein (%),30.0%,30% (target)
5,Persentase Serat (%),21.4%,35% (target)
6,Persentase Gula (%),70.0%,≤10% (target)
7,Variasi Protein,5/7 jenis,Maksimal 7/7
8,Fitness Score,0.4433,Maksimal 1.0



NUTRISI HARIAN DETAIL


,Hari,Protein (g),Serat (g),Kalsium (mg),Fosfor (mg),Kalori (kcal),Gula (g),Massa Total (g)
0,Senin,13.4,18.0,100560.0,500.0,241.6,47.0,90.1
1,Selasa,11.4,19.0,100730.0,510.0,217.6,43.0,90.1
2,Rabu,11.0,21.4,100750.0,570.0,196.0,38.0,90.1
3,Kamis,25.0,20.4,100415.0,870.0,248.0,37.0,90.1
4,Jumat,29.0,17.0,700.0,1500.0,324.0,52.0,90.0
5,Sabtu,27.6,22.0,100560.0,840.0,238.4,32.0,90.1
6,Minggu,14.0,17.0,680.0,740.0,284.0,57.0,90.0


In [100]:
# ==============================================================================
# V. VISUALISASI DAN SAVE OUTPUT
# ==============================================================================

# Create output directory and timestamp
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
plots_dir = f'{output_dir}/plots_{timestamp}'
os.makedirs(plots_dir, exist_ok=True)
print(f"\nMenyimpan laporan dan grafik di folder: {plots_dir} dan {output_dir}")

# Set style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# 1. Evolution of fitness
plt.figure(figsize=(12, 6))
generations = range(len(best_fitness_history))
plt.plot(generations, best_fitness_history, 'b-', linewidth=2, label='Best Fitness')
plt.plot(generations, avg_fitness_history, 'r--', linewidth=1, label='Average Fitness')
plt.title('Evolusi Fitness Algoritma Genetika')
plt.xlabel('Generasi')
plt.ylabel('Fitness Score')
plt.legend()
plt.grid(True, alpha=0.3)
plt.savefig(f'{plots_dir}/01_evolusi_fitness.png', dpi=300, bbox_inches='tight')
plt.close()

# 2. Nutrition per day (Calories)
plt.figure(figsize=(10, 5))
x_pos = np.arange(len(days))
bars = plt.bar(x_pos, df_nutrition['Kalori (kcal)'], color='orange', alpha=0.7)
plt.title('Kalori per Hari')
plt.xlabel('Hari')
plt.ylabel('Kalori (kcal)')
plt.xticks(x_pos, days)
plt.axhline(y=df_nutrition['Kalori (kcal)'].mean(), color='blue', linestyle='-.', label='Rata-rata')
plt.axhline(y=65, color='red', linestyle='--', label='Target Rata-rata (65 kcal)')
plt.legend()
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height + 0.5, f'{height:.1f}', ha='center', va='bottom', fontsize=9)
plt.savefig(f'{plots_dir}/02_kalori_per_hari.png', dpi=300, bbox_inches='tight')
plt.close()

# 3. Ca:P ratio per day
plt.figure(figsize=(10, 5))
ca_p_ratios_daily = [row['Kalsium (mg)']/row['Fosfor (mg)'] if row['Fosfor (mg)'] > 0 else 0 for _, row in df_nutrition.iterrows()]
bars = plt.bar(days, ca_p_ratios_daily, color='lightblue', alpha=0.7)
plt.title('Rasio Kalsium:Fosfor (Ca:P) per Hari')
plt.ylabel('Rasio Ca:P')
plt.axhline(y=2.0, color='red', linestyle='--', label='Target (2:1)')
plt.axhline(y=1.5, color='green', linestyle=':', label='Batas Bawah Ideal (1.5:1)')
plt.axhline(y=1.0, color='gray', linestyle=':', label='Batas Minimal (1:1)')
plt.xticks(rotation=45)
plt.legend(loc='upper right')
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height + 0.05, f'{height:.2f}', ha='center', va='bottom', fontsize=9)
plt.ylim(0, max(max(ca_p_ratios_daily) * 1.1, 2.5))
plt.savefig(f'{plots_dir}/03_rasio_ca_p.png', dpi=300, bbox_inches='tight')
plt.close()

# Save Excel file with multiple sheets
excel_filename = f'{output_dir}/jadwal_makanan_sugar_glider_{timestamp}.xlsx'
with pd.ExcelWriter(excel_filename, engine='openpyxl') as writer:
    df_schedule.to_excel(writer, sheet_name='Jadwal Makanan', index=False)
    df_nutrition.round(2).to_excel(writer, sheet_name='Nutrisi Harian', index=False)
    df_summary.to_excel(writer, sheet_name='Ringkasan Nutrisi', index=False)

    df_fitness = pd.DataFrame({
        'Generasi': range(len(best_fitness_history)),
        'Best Fitness': best_fitness_history,
        'Average Fitness': avg_fitness_history
    })
    df_fitness.to_excel(writer, sheet_name='Evolusi Fitness', index=False)

print(f"\n✅ Laporan Excel dan data evolusi disimpan di: {excel_filename}")
print(f"✅ 3 Grafik utama telah disimpan di folder: {plots_dir}")


Menyimpan laporan dan grafik di folder: ./output/plots_20251105_235507 dan ./output

✅ Laporan Excel dan data evolusi disimpan di: ./output/jadwal_makanan_sugar_glider_20251105_235507.xlsx
✅ 3 Grafik utama telah disimpan di folder: ./output/plots_20251105_235507
